### From articles "Options for smart investor"

https://dzen.ru/id/5f6b2336d03ed90da6bf9d00


In [1]:
# Prepare data and option_lib to use in Google Collab
!bash -c '\
OPT_LIB_PATH="/content/option_lib"; \
if [[ ! (-z "${COLAB_JUPYTER_IP}" || -d "${OPT_LIB_PATH}" ) ]]; then \
  git clone https://github.com/akumidv/option_lib.git "${OPT_LIB_PATH}"; \
  DATA_PATH="/content/data"; \
  mkdir -p "${DATA_PATH}/DERIBIT/BTC-USD/EOD/futures"; \
  mkdir -p "${DATA_PATH}/DERIBIT/BTC-USD/EOD/options"; \
  gdown --fuzzy "https://drive.google.com/file/d/1n2T4jBHeanGLtBLlLWgYIHzK9hYsGT97/view?usp=sharing"  -O "/content/data/DERIBIT/BTC-USD/EOD/futures/2025.parquet" ;\
  gdown --fuzzy "https://drive.google.com/file/d/10Eo_4oNSlx0rbe2efn5OXToONAi5nPhV/view?usp=sharing"  -O "/content/data/DERIBIT/BTC-USD/EOD/options/2025.parquet"  ;\
  export VENV_PATH="${OPT_LIB_PATH}/.venv";\
  pip install poetry ;\
  cd ${OPT_LIB_PATH} ;\
  POETRY_VIRTUALENVS_IN_PROJECT=true poetry install --no-interaction;\
fi'
import sys, os

if os.path.isdir('/content/option_lib') and '/content/option_lib/src' not in sys.path:
    sys.path.extend(['/content/option_lib/src', '/content/option_lib/.venv/'])  # Not work yet - poetry install somewhere else
    os.environ['DATA_PATH'] = '/content/data'


In [2]:
import datetime
import pandas as pd
from chart_studio import plotly

from plotly import graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#import plotly.express as px

from option_lib.provider import PandasLocalFileProvider, RequestParameters
from option_lib.entities import OptionType, OptionPriceStatus
from option_lib import Option
from option_lib.entities import LegType, OptionLeg

init_notebook_mode(connected=True)

pd.set_option("display.max_rows", 20, "display.max_columns", 30)

In [3]:
deribit_provider = PandasLocalFileProvider('DERIBIT', os.environ.get('DATA_PATH', '../../data'))
cur_dt = datetime.date.today()
provider_params = RequestParameters(period_to=cur_dt.year)
btc = Option(deribit_provider, 'BTC', provider_params,
             option_columns=['timestamp', 'expiration_date', 'strike', 'option_type', 'price', 'underlying_expiration_date', 'exchange_price', 'exchange_iv', 'underlying_price']) #, 'iv', 'delta', 'gamma', 'vega', 'theta', 'quick_delta', 'contract_size'])


In [4]:
btc.df_hist

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price
96,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.575306,2025-02-12 00:00:00+00:00,5.254928,60.36,95757.81054
97,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.604906,2025-02-12 00:00:00+00:00,0.000000,62.78,96049.03000
99,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,p,2058.690790,2025-02-12 00:00:00+00:00,4247.351996,60.36,95757.81054
100,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,p,2065.054790,2025-02-12 00:00:00+00:00,3951.033792,62.78,96049.03000
102,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,101000.0,c,9.575306,2025-02-12 00:00:00+00:00,1.639292,63.79,95757.81054
...,...,...,...,...,...,...,...,...,...
12397,2025-02-21 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4085.454875,2025-03-07 00:00:00+00:00,4653.977163,42.07,96305.61000
12398,2025-02-22 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4104.510600,2025-03-07 00:00:00+00:00,4348.096403,42.86,96710.74000
12399,2025-02-23 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4091.734250,2025-03-07 00:00:00+00:00,4406.783346,43.39,96468.32000
12400,2025-02-24 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,6903.768320,2025-03-07 00:00:00+00:00,7965.533140,44.42,91570.16000


In [5]:
df_fut = btc.df_fut
df_fut.head(2)

,timestamp,expiration_date,price
0,2025-02-11 00:00:00+00:00,2025-02-14 00:00:00+00:00,95800.0
1,2025-02-12 00:00:00+00:00,2025-02-14 00:00:00+00:00,97942.5


In [6]:
btc.enrichment.add_future().add_intrinsic_and_time_value()
btc.df_hist.head(2)

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value
96,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.575306,2025-02-12 00:00:00+00:00,5.254928,60.36,95757.81054,0.0,9.575306
97,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.604906,2025-02-12 00:00:00+00:00,0.000000,62.78,96049.03000,0.0,9.604906


In [7]:
btc.enrichment.add_atm_itm_otm()
btc.df_hist[btc.df_hist['price_status'] == OptionPriceStatus.ATM.code].head(2)

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
208,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,96000.0,c,526.641830,2025-02-12 00:00:00+00:00,338.273538,38.81,95757.81054,0.00,526.641830,atm
209,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,96000.0,c,57.629436,2025-02-12 00:00:00+00:00,50.398863,27.13,96049.03000,49.03,8.599436,atm


In [8]:
df_opt = btc.df_hist
df_opt

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
96,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.575306,2025-02-12 00:00:00+00:00,5.254928,60.36,95757.81054,0.00000,9.575306,otm
97,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,c,9.604906,2025-02-12 00:00:00+00:00,0.000000,62.78,96049.03000,0.00000,9.604906,otm
99,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,p,2058.690790,2025-02-12 00:00:00+00:00,4247.351996,60.36,95757.81054,4242.18946,-2183.498670,itm
100,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,100000.0,p,2065.054790,2025-02-12 00:00:00+00:00,3951.033792,62.78,96049.03000,3950.97000,-1885.915210,itm
102,2025-02-11 00:00:00+00:00,2025-02-12 00:00:00+00:00,101000.0,c,9.575306,2025-02-12 00:00:00+00:00,1.639292,63.79,95757.81054,0.00000,9.575306,otm
...,...,...,...,...,...,...,...,...,...,...,...,...
12397,2025-02-21 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4085.454875,2025-03-07 00:00:00+00:00,4653.977163,42.07,96305.61000,2694.39000,1391.064875,itm
12398,2025-02-22 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4104.510600,2025-03-07 00:00:00+00:00,4348.096403,42.86,96710.74000,2289.26000,1815.250600,itm
12399,2025-02-23 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,4091.734250,2025-03-07 00:00:00+00:00,4406.783346,43.39,96468.32000,2531.68000,1560.054250,itm
12400,2025-02-24 00:00:00+00:00,2025-03-07 00:00:00+00:00,99000.0,p,6903.768320,2025-03-07 00:00:00+00:00,7965.533140,44.42,91570.16000,7429.84000,-526.071680,itm


In [9]:
# search for longest timeseries expiration_date
settlement_date = df_opt['timestamp'].max() # 2025-02-28 00:00:00+0000
expiration_date = btc.chain.get_settlement_longest_period_expired_date(settlement_date)
settlement_date, expiration_date

(Timestamp('2025-02-25 00:00:00+0000', tz='UTC'),
 Timestamp('2025-02-28 00:00:00+0000', tz='UTC'))

In [10]:
df_opt_chain = btc.chain.select_chain(settlement_date, expiration_date)

df_opt_chain

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
8988,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,100000.0,c,110.331468,2025-02-28 00:00:00+00:00,104.521597,55.17,92004.13,0.00,110.331468,otm
9003,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,100000.0,p,8182.917210,2025-02-28 00:00:00+00:00,8092.723074,55.17,92006.37,7993.63,189.287210,itm
9018,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,101000.0,c,91.942890,2025-02-28 00:00:00+00:00,78.415333,57.08,92004.13,0.00,91.942890,otm
9033,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,101000.0,p,8964.431775,2025-02-28 00:00:00+00:00,9065.927531,57.08,92006.37,8993.63,-29.198225,itm
9048,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,102000.0,c,64.360023,2025-02-28 00:00:00+00:00,59.634158,59.02,92006.37,0.00,64.360023,otm
...,...,...,...,...,...,...,...,...,...,...,...,...
10293,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,97000.0,p,5838.373515,2025-02-28 00:00:00+00:00,5252.986006,49.38,92004.13,4995.87,842.503515,itm
10308,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,98000.0,c,174.691491,2025-02-28 00:00:00+00:00,188.056309,51.08,92004.13,0.00,174.691491,otm
10323,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,98000.0,p,6527.945190,2025-02-28 00:00:00+00:00,6177.636343,51.08,92004.13,5995.87,532.075190,itm
10338,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,99000.0,c,183.885780,2025-02-28 00:00:00+00:00,135.840103,52.82,92006.37,0.00,183.885780,otm


Futures types:
* Currency
* Index
* Commodities
* Stock
* Percent

Is crypto new type or currency?


BRN specification https://www.cmegroup.com/markets/energy/crude-oil/brent-crude-oil.html

Option chain https://www.barchart.com/futures/quotes/CB*0/options?futuresOptionsView=merged

In [11]:
atm_nearest_strikes = btc.chain.get_atm_nearest_strikes()
atm_strike = atm_nearest_strikes[0]
atm_strike

np.float64(92000.0)

In [12]:
atm_strike_df = df_opt_chain[df_opt_chain['strike'] == atm_strike]
atm_strike_row = atm_strike_df[atm_strike_df['option_type'] == OptionType.CALL.code].iloc[0]
atm_strike_df

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
10128,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,92000.0,c,1517.057685,2025-02-28 00:00:00+00:00,1590.631305,46.86,92006.37,6.37,1510.687685,atm
10143,2025-02-25 00:00:00+00:00,2025-02-28 00:00:00+00:00,92000.0,p,1563.029130,2025-02-28 00:00:00+00:00,1584.346089,46.86,92004.13,0.00,1563.029130,atm


In [13]:
df_desk = btc.chain.get_desk(
    option_columns=['price', 'exchange_price', 'exchange_iv','price_status',
                    'intrinsic_value', 'time_value', 'timestamp', 'expiration_date', 'strike'])
res_col = ['time_value_call', 'intrinsic_value_call', 'exchange_iv_call', 'exchange_price_call', 'price_call', 'price_status_call',
            'strike',
           'price_status_put', 'price_put', 'exchange_price_put', 'exchange_iv_put', 'intrinsic_value_put', 'time_value_put',
           'timestamp', 'underlying_price', 'expiration_date', 'underlying_expiration_date']
df_desk[df_desk['strike'].isin(atm_nearest_strikes[:10])][res_col]

,time_value_call,intrinsic_value_call,exchange_iv_call,exchange_price_call,price_call,price_status_call,strike,price_status_put,price_put,exchange_price_put,exchange_iv_put,intrinsic_value_put,time_value_put,timestamp,underlying_price,expiration_date,underlying_expiration_date
18,360.917275,4006.37,54.08,4461.848699,4367.287275,itm,88000.0,otm,459.714450,458.319676,54.08,0.00,459.714450,2025-02-25 00:00:00+00:00,92004.13,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00
19,360.917275,4006.37,54.08,4461.848699,4367.287275,itm,88000.0,otm,459.714450,458.319676,54.08,0.00,459.714450,2025-02-25 00:00:00+00:00,92006.37,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00
20,754.156700,2004.13,49.32,2845.006314,2758.286700,itm,90000.0,otm,919.428900,840.098736,49.32,0.00,919.428900,2025-02-25 00:00:00+00:00,92004.13,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00
21,754.156700,2004.13,49.32,2845.006314,2758.286700,itm,90000.0,otm,919.428900,840.098736,49.32,0.00,919.428900,2025-02-25 00:00:00+00:00,92006.37,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00
22,1202.499360,1004.13,47.68,2159.055350,2206.629360,itm,91000.0,otm,1149.286125,1153.458493,47.68,0.00,1149.286125,2025-02-25 00:00:00+00:00,92004.13,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00
23,1202.499360,1004.13,47.68,2159.055350,2206.629360,itm,91000.0,otm,1149.286125,1153.458493,47.68,0.00,1149.286125,2025-02-25 00:00:00+00:00,92006.37,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00
24,1510.687685,6.37,46.86,1590.631305,1517.057685,atm,92000.0,atm,1563.029130,1584.346089,46.86,0.00,1563.029130,2025-02-25 00:00:00+00:00,92004.13,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00
25,1510.687685,6.37,46.86,1590.631305,1517.057685,atm,92000.0,atm,1563.029130,1584.346089,46.86,0.00,1563.029130,2025-02-25 00:00:00+00:00,92006.37,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00
26,1149.286125,0.00,46.62,1140.793360,1149.286125,otm,93000.0,itm,2114.686470,2133.817947,46.62,993.63,1121.056470,2025-02-25 00:00:00+00:00,92004.13,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00
27,1149.286125,0.00,46.62,1140.793360,1149.286125,otm,93000.0,itm,2114.686470,2133.817947,46.62,993.63,1121.056470,2025-02-25 00:00:00+00:00,92006.37,2025-02-28 00:00:00+00:00,2025-02-28 00:00:00+00:00


The time value of the option decreases by the expiration date

ITM options have a low time value (low weight) in the total option price. In fact, the deeper an option is in the money, the lower its time value and the more it resembles a linear underlying asset (i.e., the corresponding futures).

OTM options have no intrinsic value (which is understandable – they are unprofitable to execute). The entire value of an out–of-money option is the intrinsic value. It is important to understand that if the market does not overcome the strike of this option, the final expiration option price will be zero.

ATM options have the maximum weight of the time value in the option price and the maximum non-linearity. For this reason, they are the most liquid on the market. When the underlying asset moves up or down from current levels, the liquidity of these options will decrease.

In [14]:
settlement_date, expiration_date = btc.chain.get_settlement_and_expiration_date()
settlement_date, expiration_date

(Timestamp('2025-02-25 00:00:00+0000', tz='UTC'),
 Timestamp('2025-02-28 00:00:00+0000', tz='UTC'))

In [15]:
btc.chart.init()
btc.chart.price.time_values(expiration_date=expiration_date, name='ATM')
btc.chart.show()

In [16]:
#btc.chart.init()
btc.chart.price.time_values_for_strike(expiration_date=expiration_date, strike=atm_nearest_strikes[5], name=f'strike {atm_nearest_strikes[3]}')
btc.chart.price.time_values_for_distance(expiration_date=expiration_date, distance=2, name='ATM distance 1')
btc.chart.show()

In [17]:
df_time_value_strike = btc.analytic.price.time_value_series_by_strike_to_atm_distance(strike=atm_nearest_strikes[5])
df_time_value_strike.iloc[[0, int(len(df_time_value_strike) / 2), -1]]

,timestamp,strike,time_value
0,2025-02-22 00:00:00+00:00,100000.0,270.414816
50,2025-02-24 00:00:00+00:00,95000.0,18.288128
92,2025-02-25 00:00:00+00:00,95000.0,9.194289


In [18]:
df_time_value_atm = btc.analytic.price.time_value_series_by_atm_distance(distance=0)
df_time_value_atm

,timestamp,strike,time_value
17,2025-02-22 00:00:00+00:00,97000.0,1062.343920
23,2025-02-23 00:00:00+00:00,96000.0,952.502524
52,2025-02-24 00:00:00+00:00,92000.0,365.762560
85,2025-02-25 00:00:00+00:00,92000.0,229.857225


In [19]:
btc.chart.init()
btc.chart.price.time_values([df_time_value_strike, df_time_value_atm], [f'Strike cur ATM {atm_strike}', 'ATM'], expiration_date=expiration_date)
btc.chart.show()

## Non-linearity of options

In [38]:
strikes = btc.chain.get_atm_nearest_strikes()
max_strike_risk_2 = strikes[15]
strike_start = sorted(filter(lambda x: x <= atm_strike * 0.95, strikes))[-1]
strike_end = sorted(filter(lambda x: x >= max_strike_risk_2 * 1.1, strikes))[0]
strike_start, strike_end

(np.float64(86000.0), np.float64(114000.0))

In [39]:
fut_legs = [OptionLeg(strike=0, lots=1, type=LegType.FUTURE)]
fut_risk_profile, fut_risk_legs_pnl = btc.analytic.risk.chain_risk_profile(fut_legs)
fut_risk_profile = fut_risk_profile.loc[strike_start:strike_end]
fut_risk_profile.head(2)

,risk_pnl
strike,
86000.0,-6004.13
88000.0,-4006.37


In [40]:
call_legs = [OptionLeg(strike=atm_strike, lots=1, type=LegType.OPTION_CALL)]
call_risk_profile, call_risk_legs_pnl = btc.analytic.risk.chain_risk_profile(call_legs)
call_risk_profile = call_risk_profile.loc[strike_start:strike_end]
call_risk_profile.head(2)

,risk_pnl
strike,
86000.0,-1517.057685
88000.0,-1517.057685


In [43]:
call_legs_1 = [OptionLeg(strike=max_strike_risk_2, lots=1, type=LegType.OPTION_CALL)]
call_risk_profile_1, _ = btc.analytic.risk.chain_risk_profile(call_legs_1)
call_risk_profile_1 = call_risk_profile_1.loc[strike_start:strike_end]
call_risk_profile_1.head(2)

,risk_pnl
strike,
86000.0,-64.360023
88000.0,-64.360023


In [44]:
data = [
    go.Scatter(x=fut_risk_profile.index.to_list(), y=fut_risk_profile['risk_pnl'].to_list(), mode='lines', name='fut'),
    go.Scatter(x=call_risk_profile.index.to_list(), y=call_risk_profile['risk_pnl'].to_list(), mode='lines',
               name=f'call atm {call_legs[0].strike}'),
    go.Scatter(x=call_risk_profile_1.index.to_list(), y=call_risk_profile_1['risk_pnl'].to_list(), mode='lines',
               name=f'call {call_legs_1[0].strike}')
]  # markers, lines
iplot(data)